# Chapter 15. More About Type Hints

"Tôi đã học được một bài học đau đớn rằng đối với các chương trình nhỏ, gõ động là rất tốt. Đối với các chương trình lớn, bạn cần một cách tiếp cận có kỷ luật hơn. Và sẽ hữu ích nếu ngôn ngữ mang lại cho bạn kỷ luật đó"

The main topics are:

- Overloaded function signatures

- typing.TypedDict for type hinting dicts used as records

- Type casting

- Runtime access to type hints

- Generic types

    - Declaring a generic class

    - Variance: invariant, covariant, and contravariant types

    - Generic static protocols

## Overloaded Signatures

Python functions có thể truyền tham số theo các cách khác nhau via `@typing.overload`

-> Nó quan trọng vì type return phụ thuộc vào type của các tham số

In [1]:
help(sum)

Help on built-in function sum in module builtins:

sum(iterable, /, start=0)
    Return the sum of a 'start' value (default: 0) plus an iterable of numbers
    
    When the iterable is empty, return the start value.
    This function is intended specifically for use with numeric values and may
    reject non-numeric types.



`sum` được viết bằng C, nhưng `typeshed` đã overload type hints cho nó ở trong  [builtins.pyi](https://github.com/python/typeshed/blob/a8834fcd46339e17fc8add82b5803a1ce53d3d60/stdlib/2and3/builtins.pyi#L1434-L1437)

```python
@overload
def sum(__iterable: Iterable[_T]) -> Union[_T, int]: ...
@overload
def sum(__iterable: Iterable[_T], start: _S) -> Union[_T, _S]: ...
```

`__iterable` means you can call sum(my_list), but not sum(__iterable = my_list).

Can also use `@overload` in a regular Python module, trước khi func được impl

In [2]:
# Example 15-1. mysum.py: definition of the sum function with overloaded signatures

import functools
import operator
from collections.abc import Iterable
from typing import overload, Union, TypeVar

T = TypeVar('T')
S = TypeVar('S')

@overload
def sum(it: Iterable[T]) -> Union[T, int]: ... #2
@overload
def sum(it: Iterable[T], /, start: S) -> Union[T, S]: ... #3
def sum(it, /, start=0): #4
    return functools.reduce(operator.add, it, start)

#2: Cho case sum(my_iterable). Return value có thể là T - type của phần tử trong my_iterable, hoặc là int nếu my_iterable rỗng, vì start default là 0

#3: Vì có start, nên phải có type là S, vì nếu reuse T type, thì start phải cùng T type

Check [typeshed](https://github.com/python/typeshed/blob/a8834fcd46339e17fc8add82b5803a1ce53d3d60/stdlib/2and3/builtins.pyi), nó có rất nhiều ví dụ về việc dùng `@overload`

"Sometimes it’s better to be pragmatic and leave a piece of code without type hints."

### Max Overload

https://github.com/python/typeshed/issues/4051

To fix issue #4051, wrote the code in Example 15-3

In [6]:
# Example 15-3. mymax.py: top of the module, with imports, definitions, and overloads

from collections.abc import Callable, Iterable
from typing import Protocol, Any, TypeVar, overload, Union

class SupportsLessThan(Protocol):
    def __lt__(self, other: Any) -> bool: ...

T = TypeVar('T')
LT = TypeVar('LT', bound=SupportsLessThan)
DT = TypeVar('DT')

MISSING = object()
EMPTY_MSG = 'max() arg is an empty sequence'

@overload
def max(__arg1: LT, __arg2: LT, *args: LT, key: None = ...) -> LT:
    ...
@overload
def max(__arg1: T, __arg2: T, *args: T, key: Callable[[T], LT]) -> T:
    ...
@overload
def max(__iterable: Iterable[LT], *, key: None = ...) -> LT:
    ...
@overload
def max(__iterable: Iterable[T], *, key: Callable[[T], LT]) -> T:
    ...
@overload
def max(__iterable: Iterable[LT], *, key: None = ...,
        default: DT) -> Union[LT, DT]:
    ...
@overload
def max(__iterable: Iterable[T], *, key: Callable[[T], LT],
        default: DT) -> Union[T, DT]:
    ...

def max(first, *args, key=None, default=MISSING):
    if args:
        series = args
        candidate = first
    else:
        series = iter(first)
        try:
            candidate = next(series)
        except StopIteration:
            if default is not MISSING:
                return default
            raise ValueError(EMPTY_MSG) from None
    if key is None:
        for current in series:
            if candidate < current:
                candidate = current
    else:
        candidate_key = key(candidate)
        for current in series:
            current_key = key(current)
            if candidate_key < current_key:
                candidate = current
                candidate_key = current_key
    return candidate

Thanks to `duck typing`, code has no `isinstance` checks, and provides the same error checking as those type hints—but only at runtime, of course.


### Arguments implementing SupportsLessThan, but key and default not provided

The return type of max is the same as the actual arguments or items

In [7]:
"""
@overload
def max(__arg1: LT, __arg2: LT, *_args: LT, key: None = ...) -> LT:
    ...
# ... lines omitted ...
@overload
def max(__iterable: Iterable[LT], *, key: None = ...) -> LT:
    ...
"""
max(1, 2, -3)

2

In [8]:
max(['Go', 'Python', 'Rust'])

'Rust'

### Argument key provided, but no default

In [9]:
"""
@overload
def max(__arg1: T, __arg2: T, *_args: T, key: Callable[[T], LT]) -> T:
    ...
# ... lines omitted ...
@overload
def max(__iterable: Iterable[T], *, key: Callable[[T], LT]) -> T:
    ...
"""

max(1, 2, -3, key=abs)

-3

In [10]:
max(['Go', 'Python', 'Rust'], key=len)

'Python'

### Argument default provided, but no key

In [11]:
"""
@overload
def max(__iterable: Iterable[LT], *, key: None = ...,
        default: DT) -> Union[LT, DT]:
    ...
"""

max([1, 2, -3], default=0)

2

In [12]:
max([], default=None)

### Arguments key and default provided

In [13]:
"""
@overload
def max(__iterable: Iterable[T], *, key: Callable[[T], LT],
        default: DT) -> Union[T, DT]:
    ...
"""
max([1, 2, -3], key=abs, default=None)

-3

In [14]:
max([], key=abs, default=None)

### Bài học từ Overloading max

Type hints allow Mypy to flag a call like `max([None, None])` with this error message

`mymax_demo.py:109: error: Value of type variable "_LT" of "max"
  cannot be "None"`
--compli` time

In [15]:
max([None, None]) # runtime

TypeError: '<' not supported between instances of 'NoneType' and 'NoneType'

Nhưng nó phải viết quá nhiều code để support type checker, và với hàm `min` , ta cũng cần viết số code đó

OK, giờ đến phần `TypedDict`, nó cũng ko quá hữu ích, và nó chỉ cho ta thấy những hạn chế của `static typing` khi handle dynamic structure (VD: JSON)

## TypedDict

Rất OK nếu dùng `TypeDict` để tránh errors khi handle dynamic structure như JSON API response.

Nhưng correct handle JSON phải đc done ở runtime, chứ ko phải ở static type checking ở type hints.

Để runtime check cho JSON/XML structure bằng type hints, nên dùng `pydantic`

Ví dụ:

```json
{"isbn": "0134757599",
 "title": "Refactoring, 2e",
 "authors": ["Martin Fowler", "Kent Beck"],
 "pagecount": 478}
```

Type hints, có 3 cách để biểu đạt
- `Dict[str, Any]`
- `Dict[str, Union[str, int, List[str]]]`
- `TypedDict` (OK hơn 2 cái trên): khá giống với `typing.NamedTuple` (chap 5) nhưng chỉ dùng cho type checker, no runtime effect

In [16]:
from typing import TypedDict

class BookDict(TypedDict):
    isbn: str
    title: str
    authors: list[str]
    pagecount: int

pp = BookDict(title='Programming Pearls',
              authors='Jon Bentley',
              isbn='0201657880',
              pagecount=256)

type(pp)

dict

In [17]:
pp.title # vì là dict -> ko thể dùng object.field

AttributeError: 'dict' object has no attribute 'title'

In [18]:
pp['title']

'Programming Pearls'

In [19]:
BookDict.__annotations__

{'isbn': str, 'title': str, 'authors': list[str], 'pagecount': int}

Example 15-6 builds a valid `BookDict` and tries some operations on it. This shows how `TypedDict` enables `Mypy` to catch errors, shown in Example 15-7.

In [21]:
# Example 15-6. demo_books.py
from typing import TYPE_CHECKING

def demo() -> None:  # <1>
    book = BookDict(  # <2> valid BookDict
        isbn='0134757599',
        title='Refactoring, 2e',
        authors=['Martin Fowler', 'Kent Beck'],
        pagecount=478
    )
    authors = book['authors'] # <3> Mypy will infer the type of authors
    if TYPE_CHECKING:  # <4>
        reveal_type(authors)  # <5>
    authors = 'Bob'  # <6>
    book['weight'] = 4.2
    del book['title']


if __name__ == '__main__':
    demo()

# 1: Remember to add a return type, so that Mypy doesn’t ignore the function.
# 4: typing.TYPE_CHECKING is only True when the program is being type checked. At runtime, it’s always false.
# 5: reveal_type is not a runtime Python function, but a debugging facility provided by Mypy
# 6: Error

```bash
…/typeddict/ $ mypy demo_books.py
demo_books.py:13: note: Revealed type is 'built-ins.list[built-ins.str]'  1
demo_books.py:14: error: Incompatible types in assignment
                  (expression has type "str", variable has type "List[str]")  2
demo_books.py:15: error: TypedDict "BookDict" has no key 'weight'  3
demo_books.py:16: error: Key 'title' of TypedDict "BookDict" cannot be deleted  4
Found 3 errors in 1 file (checked 1 source file)
```

#1. This note is the result of reveal_type(authors).
#2. authors lấy type từ `book['authors']`: List[str]
#3. Book ko có key là weight

Giờ thì cần sinh ra XML từ book record

Static type checking is unable to prevent errors with code that is dynamic, such as `json.loads()`, which builds Python objects of different types at runtime, as Examples 15-11, 15-12, and 15-13 demonstrate.

- Example 15-11. demo_not_book.py: from_json returns an invalid BookDict, and to_xml accepts it

```bash
…/typeddict/ $ mypy demo_not_book.py
demo_not_book.py:12: note: Revealed type is
   'TypedDict('books.BookDict', {'isbn': built-ins.str,
                                 'title': built-ins.str,
                                 'authors': built-ins.list[built-ins.str],
                                 'pagecount': built-ins.int})'  1
demo_not_book.py:13: note: Revealed type is 'built-ins.list[built-ins.str]'  2
demo_not_book.py:16: error: TypedDict "BookDict" has no key 'flavor'  3
Found 1 error in 1 file (checked 1 source file)
```


```bash
…/typeddict/ $ python3 demo_not_book.py
{'title': 'Andromeda Strain', 'flavor': 'pistachio', 'authors': True}  1
pistachio  2
<BOOK>  3
        <TITLE>Andromeda Strain</TITLE>
        <FLAVOR>pistachio</FLAVOR>
        <AUTHORS>True</AUTHORS>
</BOOK>
```
1. Nó ko là `BookDict`
2. Giá trị của `not_book['flavor']`
3. `to_xml` takes a BookDict argument, but there is no runtime checking: garbage in, garbage out.

--> compli time sai, nhưng runtime vẫn ok
--> `TypeDict` để handle `JSON`/`XML` ko an toàn
--> Khi đó, nên dùng `pydantic`: https://pypi.org/project/pydantic/

```python
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name = 'John Doe'
    signup_ts: Optional[datetime] = None
    friends: List[int] = []

external_data = {'id': '123', 'signup_ts': '2017-06-01 12:22', 'friends': [1, '2', b'3']}
user = User(**external_data)
print(user)
#> User id=123 name='John Doe' signup_ts=datetime.datetime(2017, 6, 1, 12, 22) friends=[1, 2, 3]
print(user.id)
#> 123
```

## Type Casting

Nên tránh dùng nhất khi có thể

The `typing.cast()` special function provides one way to handle type checking malfunctions (trục trặc) or incorrect type hints in code we can’t fix.

At runtime, typing.cast does absolutely nothing.

In [23]:
def cast(typ, val):
    """Cast a value to a type.
    This returns the value unchanged.  To the type checker this
    signals that the return value has the designated type, but at
    runtime we intentionally don't check anything (we want this
    to be as fast as possible).
    """
    return val

In [24]:
# Ví dụ cần dùng cast,
# VD1: tìm str đầu tiên trong list

from typing import cast

def find_first_str(a: list[object]) -> str:
    index = next(i for i, x in enumerate(a) if isinstance(x, str))
    # We only get here if there's at least one string
    return cast(str, a[index])

# VD2: Mypy reported this error
addr = server.sockets[0].getsockname()
## Mypy error: Value of type "Optional[List[socket]]" is not indexable

## --> dùng cast
from asyncio.trsock import TransportSocket
from typing import cast

# ... many lines omitted ...

socket_list = cast(tuple[TransportSocket, ...], server.sockets)
addr = socket_list[0].getsockname()

WARNING

Don’t get too comfortable using `cast` to silence `Mypy`, because Mypy is usually right when it reports an error. If you are using cast very often, that’s a `code smell`.

Of course, not all typing mishaps can be fixed with `cast`. Sometimes we need `# type: ignore`, the occasional `Any`, or even leaving a function without type hints.

## Reading Type Hints at Runtime

At import time, Python reads the type hints in functions, classes, and modules, and stores them in attributes named `__annotations__`

In [6]:
def clip(text: str, max_len: int = 80) -> str:
    a = text + max_len

In [7]:
clip.__annotations__ # <-- evaluated by the interpreter at import time

{'text': str, 'max_len': int, 'return': str}

### Problems with Annotations at Runtime

Việc sử dụng type hints có 2 vấn đề:
- Importing modules tốn nhiều CPU + Memory khi sử dụng type hints
- Các loại chưa được define sẽ sử dụng string ('int') thay vì actual type (int)
   - Lưu annotations là strings được dùng do vấn đề “forward reference” (when a type hint needs to refer to a class defined below in the same module)
   - Code needs to figure out what that string means: `typing.get_type_hints`

In [ ]:
# Ví dụ: Kiểu Rectangle
# Vì __annotations__ đc xác định ở import time
class Rectangle:
    # ... lines omitted ...
    def stretch(self, factor: float) -> 'Rectangle':
        return Rectangle(width=self.width * factor)

```
from __future__ import annotations
>>> from clip_annot_post import clip
>>> clip.__annotations__
{'text': 'str', 'max_len': 'int', 'return': 'str'}
```

```
>>> from clip_annot_post import clip
>>> from typing import get_type_hints
>>> get_type_hints(clip)
{'text': <class 'str'>, 'max_len': <class 'int'>, 'return': <class 'str'>}
```

-->

Calling `get_type_hints` gives us the `real types—even` in some cases where the `original type hint` is written as a quoted string. That’s the recommended way to read type hints at runtime.

NOTE
Các công ty sử dụng Python ở quy mô rất lớn muốn có những lợi ích của static typing, nhưng họ không muốn trả giá cho việc đánh giá các type hint ở import time. Static check thường ở máy local hoặc CI, nó check thường xuyên -> chi phí lớn.

-> Có người thì thích type hint ở dạng String (giảm loading cost) , Có người lại thích type hint ở runtime, lưu type objects  (VD: người tạo và người dùng pydantic và FastAPI)
- FastAPI (code API nhanh mà dễ ẹc) được build dựa trên OpenAPI (trước có tên Swagger), phần web được support bởi Starlette, còn phần data được support bởi Pydantic.

### Dealing with the Problem

Nếu muốn đọc annotations ở runtime, nên:
- Avoid reading `__annotations__` directly; instead, use:
   - `inspect.get_annotations` (from Python 3.10)
   - `typing.get_type_hints` (since Python 3.5)
- Write a custom function of your own as a thin wrapper around `inspect.get_annotations` or `typing.get_type_hints`, and have the rest of your codebase call that custom function, so that future changes are localized to a single function.

Ví dụ poin 2:

```python
class Checked:
    @classmethod
    def _fields(cls) -> dict[str, type]:
        return get_type_hints(cls)
    # ... more lines ...
```

The `Checked._fields` class method protects other parts of the module from depending directly on `typing.get_type_hints`

Các phần còn lại của chương này đề cập đến khái quát, bắt đầu với cách định nghĩa một lớp chung mà người dùng có thể tham số hóa.

## Implementing a Generic Class

Ví dụ 13-7, ta định nghĩa `Tombola` ABC: 1 interface cho class như "bingo cage". `LottoBlower` class ở ví dụ 13-10 là 1 implement của nó.

Giờ ta sẽ xem về generic version của `LottoBlower` ở Ví dụ 15-15.

Xem cách implements ở

[generic_lotto.py](https://github.com/huymq1710/example-code-2e/blob/master/15-more-types/lotto/generic_lotto.py)

Generic class cần định nghĩa kiểu

VD: `lotto = LottoBlower[int]([1,2,3])`

### Các thuật ngữ cơ bản của Generic Types

Generic type
- A type declared with one or more type variables.
- Examples: `LottoBlower[T]`, `abc.Mapping[KT, VT]`

Formal type parameter
- The type variables that appear in a generic type declaration.
- Example: KT and VT in the previous example `abc.Mapping[KT, VT]`

Parameterized type
- A type declared with actual type parameters.
- Examples: `LottoBlower[int]`, `abc.Mapping[str, float]`

Actual type parameter
- The actual types given as parameters when a parameterized type is declared.
- Example: the `int` in `LottoBlower[int]`



Chủ đề tiếp theo là về cách làm cho các generic types linh hoạt hơn, giới thiệu các khái niệm về hiệp phương sai, chống phương sai và bất biến.

## Variance (phương sai)

- Phần khó nhất
- Giống toán


NOTE

Trong thực tế, phương sai chủ yếu được dùng bởi người viết thư viện, người muốn hỗ trợ new generic container types hoặc cung cấp callback-based APIs.

Ta có thể tránh sự phức tạp bằng cách chỉ hỗ trợ invariant (bất biến) containers

Ta lần đầu thất *variance* ở chương 8 (Variance in Callable types):

In [ ]:
# Example 8-24. Minh hoạ variance.
from collections.abc import Callable

def update(  #1
    probe: Callable[[], float],  #2
    display: Callable[[float], None]  #3
    ) -> None:
    temperature = probe()
    # imagine lots of control code here
    display(temperature)

def probe_ok() -> int:  #4
    return 42

def display_wrong(temperature: int) -> None:  #5
    print(hex(temperature))

update(probe_ok, display_wrong)  # type error  #6

def display_ok(temperature: complex) -> None:  #7
    print(temperature)

update(probe_ok, display_ok)  # OK  8

#1: `update` takes two callables as arguments.
#2: `probe` must be a callable that takes no arguments and returns a float
#3: `display` takes a float argument and returns None
#4: `probe_ok` is consistent-with `Callable[[], float]` because returning an int does not break code that expects a float

#5: display_wrong is not consistent-with Callable[[float], None] because there’s no guarantee that a function that expects an int can handle a float;

#6: Mypy flags this line because display_wrong is incompatible with the type hint in the display parameter of update.

#7: display_ok is consistent-with Callable[[float], None] because a function that accepts a complex can also handle a float argument.

#8: Mypy OK

Ở đây, chúng ta sẽ mở rộng khái niệm này để bao gồm generic collection types, làm cho khái niệm trừu tượng này cụ thể hơn.

Ví dụ: Máy bán hàng ở trường chỉ được dùng loại bán nước trái cây (juice dispensers). Không được loại máy bán thường (General beverage dispensers) vì nó có thể bán đồ có cồn.

### An Invariant (bất biến) Dispenser
- Chỉ là class

Bắt đầu vs tình huống ở ví dụ bằng 1 generic `BeverageDispenser` class

In [1]:
# Example 15-18. invariant.py
# tag::BEVERAGE_TYPES[]
from typing import TypeVar, Generic

class Beverage:  # <1>
    """Any beverage (đồ uống)."""

class Juice(Beverage):
    """Any fruit juice."""

class OrangeJuice(Juice):
    """Delicious juice from Brazilian oranges."""

T = TypeVar('T')  # <2> Simple TypeVar declaration.

class BeverageDispenser(Generic[T]):  # <3> được tham số hóa trên loại đồ uống
    """A dispenser (máy bán hàng) parameterized on the beverage type."""
    def __init__(self, beverage: T) -> None:
        self.beverage = beverage

    def dispense(self) -> T:
        return self.beverage

def install(dispenser: BeverageDispenser[Juice]) -> None:  # <4> Chỉ cho install loại Juice
    """Install a fruit juice dispenser."""
# end::BEVERAGE_TYPES[]

In [3]:
# Hợp lệ:
juice_dispenser = BeverageDispenser(Juice())
install(juice_dispenser)

In [2]:
# Không hợp lệ
beverage_dispenser = BeverageDispenser(Beverage())
install(beverage_dispenser)
## mypy: Argument 1 to "install" has
## incompatible type "BeverageDispenser[Beverage]"
##          expected "BeverageDispenser[Juice]"

In [ ]:
# Nhưng cái này cũng ko hợp lệ

orange_juice_dispenser = BeverageDispenser(OrangeJuice())
install(orange_juice_dispenser)
## mypy: Argument 1 to "install" has
## incompatible type "BeverageDispenser[OrangeJuice]"
##          expected "BeverageDispenser[Juice]"

--> Ta nói `BeverageDispenser(Generic[T])` là invariant (bất biến/ko thể đổi) khi `BeverageDispenser[OrangeJuice]` không tương thích với `BeverageDispenser[Juice]`—mặc dù thực tế là `OrangeJuice` là một loại phụ của `Juice`.

Python mutable collection types—such as `list` and `set`—are invariant.

NOTE
- Immutable: bất biến, typically is applied to objects
- Invariant: bất biến, typically applied to attributes/values.

### A Covariant (Hiệp phương sai) Dispenser
- Chấp nhận class + con của nó

Nếu ta muốn linh hoạt, chấp nhận Juice và subtype của nó (OrangeJuice) -> ta làm generic class trở lên `Covariant`

In [4]:
# Example 15-19. covariant.py
from typing import TypeVar, Generic


class Beverage:
    """Any beverage."""


class Juice(Beverage):
    """Any fruit juice."""


class OrangeJuice(Juice): # Subclass của Juice
    """Delicious juice from Brazilian oranges."""


# tag::BEVERAGE_TYPES[]
T_co = TypeVar('T_co', covariant=True)  # <1> Chấp nhận class + con của nó


class BeverageDispenser(Generic[T_co]):  # <2>
    def __init__(self, beverage: T_co) -> None:
        self.beverage = beverage

    def dispense(self) -> T_co:
        return self.beverage

def install(dispenser: BeverageDispenser[Juice]) -> None:  # <3>
    """Install a fruit juice dispenser."""
# end::BEVERAGE_TYPES[]

In [5]:
# Hợp lệ
orange_juice_dispenser = BeverageDispenser(OrangeJuice())
install(orange_juice_dispenser)

### A Contravariant (trái ngược/chống lại) Trash Can
- Chấp nhận class + cha của  nó

Giờ ta sẽ lập 1 quy tắc ở quán cafe để vứt rác:
- `Refuse`:  general type of trash
- `Biodegradable`: rác có thể phân huỷ
- `Compostable`: rác có thể phân huỷ và được chuyển thành phân bón

In [ ]:
# Example 15-20. contravariant.py: type definitions and install function

# tag::TRASH_TYPES[]
from typing import TypeVar, Generic

class Refuse:  # <1>
    """Any refuse."""

class Biodegradable(Refuse):
    """Biodegradable refuse."""

class Compostable(Biodegradable):
    """Compostable refuse."""

T_contra = TypeVar('T_contra', contravariant=True)  # <2> Chấp nhận class + cha của  nó

class TrashCan(Generic[T_contra]):  # <3>
    def put(self, refuse: T_contra) -> None:
        """Store trash until dumped."""

def deploy(trash_can: TrashCan[Biodegradable]):
    """Deploy a trash can for biodegradable refuse."""
# end::TRASH_TYPES[]


################################################ contravariant trash can


# tag::DEPLOY_TRASH_CANS[]
bio_can: TrashCan[Biodegradable] = TrashCan()
deploy(bio_can)

trash_can: TrashCan[Refuse] = TrashCan()
deploy(trash_can)
# end::DEPLOY_TRASH_CANS[]


################################################ more specific trash can

# tag::DEPLOY_NOT_VALID[]
compost_can: TrashCan[Compostable] = TrashCan()
deploy(compost_can)
## mypy: Argument 1 to "deploy" has
## incompatible type "TrashCan[Compostable]"
##          expected "TrashCan[Biodegradable]"
# end::DEPLOY_NOT_VALID[]

### Variance Review

Các phần sau tóm tắt lại khái niệm về các loại invariant (bất biến), covariant (hiệp biến) và contravariant (chống biến), đồng thời cung cấp một số quy tắc để dùng.

### Rules of thumb when thinking variance

- If a formal type parameter defines a type for data that comes out of the object, it can be covariant.
- If a formal type parameter defines a type for data that goes into the object after its initial construction, it can be contravariant.
- If a formal type parameter defines a type for data that comes out of the object and the same parameter defines a type for data that goes into the object, it must be invariant.
- To err on the safe side, make formal type parameters invariant.

`Callable[[ParamType, …], ReturnType]` demonstrates rules #1 and #2: The `ReturnType` is *covariant*, and each `ParamType` is *contravariant*.

By default, `TypeVar` creates formal parameters that are *invariant*, and that’s how the mutable collections in the standard library are annotated.


## Implementing a Generic Static Protocol
Dùng ý tưởng covariance ở phần trước

```python
@runtime_checkable
class SupportsAbs(Protocol[T_co]):
    """An ABC with one abstract method __abs__ that is covariant in its
        return type."""
    # type class + con của nó
    __slots__ = ()

    @abstractmethod
    def __abs__(self) -> T_co:
        pass
```

In [6]:
# Example 15-21. abs_demo.py: use of the generic SupportsAbs protocol

import math
from typing import NamedTuple, SupportsAbs

class Vector2d(NamedTuple):
    x: float
    y: float

    def __abs__(self) -> float:  # <1>
        return math.hypot(self.x, self.y)

def is_unit(v: SupportsAbs[float]) -> bool:  # <2>
    """'True' if the magnitude of 'v' is close to 1."""
    return math.isclose(abs(v), 1.0)  # <3>

assert issubclass(Vector2d, SupportsAbs)  # <4>

v0 = Vector2d(0, 1)  # <5>
sqrt2 = math.sqrt(2)
v1 = Vector2d(sqrt2 / 2, sqrt2 / 2)
v2 = Vector2d(1, 1)
v3 = complex(.5, math.sqrt(3) / 2)
v4 = 1  # <6>

# 1: Defining __abs__ makes Vector2d consistent-with SupportsAbs.
# 2: Parameterizing SupportsAbs with float ensures…
# 3: …that Mypy accepts abs(v) as the first argument for math.isclose.

# 6: The int type is also consistent-with SupportsAbs. According to typeshed, int.__abs__ returns an int, which is consistent-with the float type parameter declared in the is_unit type hint for the v argument.

assert is_unit(v0)
assert is_unit(v1)
assert not is_unit(v2)
assert is_unit(v3)
assert is_unit(v4)

print('OK')


OK


In [7]:
v0, v1, v2, v3, v4

(Vector2d(x=0, y=1),
 Vector2d(x=0.7071067811865476, y=0.7071067811865476),
 Vector2d(x=1, y=1),
 (0.5+0.8660254037844386j),
 1)

In [8]:
# Example 15-22. generic_randompick.py: definition of generic RandomPicker

from typing import Protocol, runtime_checkable, TypeVar

T_co = TypeVar('T_co', covariant=True)  # <1>

@runtime_checkable
class RandomPicker(Protocol[T_co]):  # <2>
    def pick(self) -> T_co: ...  # <3>

# 1. Declare T_co as covariant.

# 2. This makes RandomPicker generic with a covariant formal type parameter.

# 3. Use T_co as the return type.

"""
The generic RandomPicker protocol can be covariant because its only formal parameter is used in a return type.
"""

'\nThe generic RandomPicker protocol can be covariant because its only formal parameter is used in a return type.\n'